In [1]:
import pandas as pd

df_train = pd.read_csv('train.csv').fillna("")

#Generating a graph of Questions and their neighbors
def generate_qid_graph_table(row):

    hash_key1 = row["qid1"]
    hash_key2 = row["qid2"]
        
    qid_graph.setdefault(hash_key1, []).append(hash_key2)
    qid_graph.setdefault(hash_key2, []).append(hash_key1)

qid_graph = {}
df_train.apply(generate_qid_graph_table, axis = 1); #You should apply this on df_test too. Avoiding here on the kernel.

In [ ]:
# # coding: utf-8
# # Based on notebook by https://www.kaggle.com/shubh24 
# # https://www.kaggle.com/shubh24/pagerank-on-quora-a-basic-implementation
# __author__ = 'ZFTurbo: https://kaggle.com/zfturbo'

import pandas as pd
import hashlib
import gc 

df_train = pd.read_csv('train.csv').fillna("")
df_test = pd.read_csv('test.csv').fillna("")


# Generating a graph of Questions and their neighbors
def generate_qid_graph_table(row):
    hash_key1 = hashlib.md5(row["question1"].encode('utf-8')).hexdigest()
    hash_key2 = hashlib.md5(row["question2"].encode('utf-8')).hexdigest()

    qid_graph.setdefault(hash_key1, []).append(hash_key2)
    qid_graph.setdefault(hash_key2, []).append(hash_key1)


qid_graph = {}
print('Apply to train...')
df_train.apply(generate_qid_graph_table, axis=1)
print('Apply to test...')
df_test.apply(generate_qid_graph_table, axis=1)


def pagerank():
    MAX_ITER = 20
    d = 0.85

    # Initializing -- every node gets a uniform value!
    pagerank_dict = {i: 1 / len(qid_graph) for i in qid_graph}
    num_nodes = len(pagerank_dict)

    for iter in range(0, MAX_ITER):

        for node in qid_graph:
            local_pr = 0

            for neighbor in qid_graph[node]:
                local_pr += pagerank_dict[neighbor] / len(qid_graph[neighbor])

            pagerank_dict[node] = (1 - d) / num_nodes + d * local_pr

    return pagerank_dict

print('Main PR generator...')
pagerank_dict = pagerank()

def get_pagerank_value(row):
    q1 = hashlib.md5(row["question1"].encode('utf-8')).hexdigest()
    q2 = hashlib.md5(row["question2"].encode('utf-8')).hexdigest()
    s = pd.Series({
        "q1_pr": pagerank_dict[q1],
        "q2_pr": pagerank_dict[q2]
    })
    return s

print('Apply to train...')
pagerank_feats_train = df_train.apply(get_pagerank_value, axis=1)
print('Writing train...')
pagerank_feats_train.to_csv("pagerank_train.csv", index=False)
del df_train
gc.collect()
print('Apply to test...')
pagerank_feats_test = df_test.apply(get_pagerank_value, axis=1)
print('Writing test...')
pagerank_feats_test.to_csv("pagerank_test.csv", index=False)

Apply to train...
Apply to test...
Main PR generator...
Apply to train...
Writing train...
Apply to test...
Writing test...


In [7]:
pagerank_feats_test.head()

,q1_pr,q2_pr
0,2.088105e-07,2.088105e-07
1,2.367208e-07,2.194141e-07
2,2.088105e-07,2.088105e-07
3,2.088105e-07,2.088105e-07
4,2.088105e-07,2.088105e-07


In [2]:
def pagerank():

    MAX_ITER = 20 #Let me know if you find an optimal iteration number!
    d = 0.85
    
    #Initializing -- every node gets a uniform value!
    pagerank_dict = {i:1/len(qid_graph) for i in qid_graph}
    num_nodes = len(pagerank_dict)
    
    for iter in range(0, MAX_ITER):
        
        for node in qid_graph:    
            local_pr = 0
            
            for neighbor in qid_graph[node]:
                local_pr += pagerank_dict[neighbor]/len(qid_graph[neighbor])
            
            pagerank_dict[node] = (1-d)/num_nodes + d*local_pr

    return pagerank_dict

pagerank_dict = pagerank()


In [3]:
def get_pagerank_value(row):
    return pd.Series({
        "q1_pr": pagerank_dict[row["qid1"]],
        "q2_pr": pagerank_dict[row["qid2"]]
    })

pagerank_feats_train = df_train.apply(get_pagerank_value, axis = 1);

In [4]:
pagerank_feats_train

,q1_pr,q2_pr
0,1.858968e-06,1.858968e-06
1,3.989380e-06,1.969041e-06
2,1.858968e-06,1.858968e-06
3,1.858968e-06,1.858968e-06
4,3.324574e-06,1.220808e-06
5,1.858968e-06,1.858968e-06
6,1.858968e-06,1.858968e-06
7,1.858968e-06,1.858968e-06
8,1.550948e-06,2.244366e-06
9,1.858968e-06,1.858968e-06
